In [ ]:
import fitz  # PyMuPDF for PDF parsing
import re

# llama-index imports
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core import Document, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import CompactAndRefine


pdf_path = "sample_contract.pdf"   # can change this


def process_and_index_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = "\n".join(page.get_text() for page in doc)
    document = Document(text=full_text, metadata={"source": pdf_path})
    embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
    index = VectorStoreIndex.from_documents([document], embed_model=embed_model)
    return index, [document]


llm = LlamaCPP(
    model_path="/content/mistral.gguf",
    temperature=0.0,
    max_new_tokens=200,
    context_window=4096,
)


def build_rag_pipeline(index, docs, llm):
    # Dense embedding retriever
    embed_retriever = index.as_retriever(similarity_top_k=3)
    # Sparse BM25 retriever
    bm25_retriever = BM25Retriever(documents=docs)
    # Fusion retriever: combines both and performs query expansion
    fusion_retriever = QueryFusionRetriever(
        retrievers=[embed_retriever, bm25_retriever],
        llm=llm,
        similarity_top_k=3,
        num_queries=3,
        mode="reciprocal_rerank"
    )
    synthesizer = CompactAndRefine(llm=llm, verbose=True)
    return RetrieverQueryEngine(
        retriever=fusion_retriever,
        response_synthesizer=synthesizer
    )


index, docs = process_and_index_pdf(pdf_path)
rag_engine   = build_rag_pipeline(index, docs, llm)

while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("\nChatbot: Goodbye! Have a great day!")
        break
    try:
        response = rag_engine.query(user_input)
        print(f"\nChatbot: {response}")
    except Exception as e:
        print(f"\nError: {e}")
        print("Please try again or check your setup/API key")